**Q1**)

To use Zero-Shot Learning and Few-Shot Learning for classifying human activities based on accelerometer data, we will use a large language model that can handle text input. Since accelerometer data is not text-based, we'll first convert the accelerometer data into a textual description. then do the zero shot and reconvert them into numbers. Then we can see the predicted labels

 **1) Zero shot learning**

In [ ]:
!pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import numpy as np
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import re
import matplotlib.pyplot as plt

x_test_path = '/content/X_test.npy'
y_test_path = '/content/y_test.npy'
x_train_path = '/content/X_train.npy'
y_train_path = '/content/y_train.npy'
x_test = np.load(x_test_path)
y_test = np.load(y_test_path)

label_mapping = {
    "LAYING": 1,
    "SITTING": 2,
    "STANDING": 3,
    "WALKING": 4,
    "WALKING_DOWNWARDS": 5,
    "WALKING_UPWARDS": 6
}
model_name = "llama3.1-8b"
api_key = "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"
model = ChatGroq(api_key=api_key)

In [ ]:
# Function to format a single example for zero-shot learning
def format_example(example):
    ax = example[:, 0]
    ay = example[:, 1]
    az = example[:, 2]

    ax_str = ", ".join(map(str, ax))
    ay_str = ", ".join(map(str, ay))
    az_str = ", ".join(map(str, az))

    # Simplified prompt
    formatted_string = (
        f"Classify the activity based on the following accelerometer data:\n"
        f"ax: [{ax_str}]\n"
        f"ay: [{ay_str}]\n"
        f"az: [{az_str}]\n"
        f"Activity: "
    )

    return formatted_string

# Prepare input examples from x_test
formatted_examples = [format_example(example) for example in x_test]

# Perform zero-shot prediction and print raw predictions
y_pred = [model.predict(example) for example in formatted_examples]

# Extract activity labels from the model's response
def extract_label(response):
    match = re.search(r'\b(LAYING|SITTING|STANDING|WALKING|WALKING_DOWNWARDS|WALKING_UPWARDS)\b', response.upper())
    return match.group(0) if match else "UNKNOWN"

# Convert predictions to numeric labels
y_pred_labels = [label_mapping.get(extract_label(pred), 0) for pred in y_pred]

# Print raw predictions, predicted labels, and actual labels for the first 10 samples
#print("Example\tRaw Prediction\tPredicted Label\tActual Label")
for i in range(5):
    raw_pred = y_pred[i]
    pred_label = label_mapping.get(extract_label(raw_pred), 0)
    actual_label = y_test[i]
    #print(f"{i+1}\t{raw_pred}\t{pred_label}\t{actual_label}")

# Print the numeric predictions for all samples
print("\nAll predicted labels:")
print(y_pred_labels)
print("\n All True lables:")
print(y_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"\nAccuracy: {accuracy}")



All predicted labels:
[4, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 0, 4, 0, 4, 3, 4, 2, 4, 4, 0, 4, 4, 4, 4, 2, 4, 4, 0, 4, 4, 4, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 0, 2, 4, 3, 4]

 All True lables:
[3 1 2 5 5 1 1 5 3 2 6 5 6 5 6 1 6 5 2 5 4 3 2 2 1 4 6 4 1 2 6 2 4 4 3 6 6
 3 1 5 3 2 1 4 4 4 5 1 3 3 3 6 2 4]

Accuracy: 0.14814814814814814


So, We get an accuracy around 15% in zero shot learning. For this data. We shall see what happens when we give a few examples and do the same thing.

**2) Few shot Learning**

In [ ]:
from transformers import pipeline

# Create a pipeline for zero-shot classification using XLM-R
few_shot_classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# Define a few-shot prompt
few_shot_prompt = (
    "Examples of activities with accelerometer data:\n"
    "1. ax = 0.1871044, ay = 0.7884942, az = 0.5742789: laying\n"
    "2. ax = 0.9908435, ay = -0.10484, az = -0.2516169: sitting\n"
    "3. ax = 0.9982275, ay = -0.1632168, az = 0.2099063: standing\n"
    "4. ax = 0.6913776, ay = -0.4242498, az = -0.01527766: walking\n"
    "5. ax = 0.6580693, ay = -0.26417, az = -0.117143: walking_downwards\n"
    "6. ax = 1.240858, ay = -0.1790761, az = -0.5927427: walking_upwards\n"
    "\n"
    "Classify the following accelerometer data:\n"
    "ax = 1.187, ay = -0.6640613, az = -0.5706888"
)

# Define candidate labels
candidate_labels = ["laying", "sitting", "standing", "walking", "walking_downwards", "walking_upwards"]

# Perform few-shot classification
result = few_shot_classifier(few_shot_prompt, candidate_labels)

# Extract the max score and its corresponding label
max_index = result['scores'].index(max(result['scores']))
max_score = result['scores'][max_index]
max_label = result['labels'][max_index]

print(result['labels'])
print(result['scores'])
print(f"\nMax score: {max_score}\n")
print(f"Corresponding activity: {max_label}")


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['walking_upwards', 'laying', 'walking', 'walking_downwards', 'standing', 'sitting']
[0.3513725697994232, 0.16650117933750153, 0.16574865579605103, 0.14687234163284302, 0.09441468119621277, 0.07509055733680725]

Max score: 0.3513725697994232

Corresponding activity: walking_upwards


The output is not correct, but it is very close in saying the output as actual output has 2nd most probability of predicting the output. This is due to the examples we give to few shot learning, it can learn better.

Hence Few shots learning is better than Zero shot learning.

**Q3**